In [78]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key= os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7549487b6d70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7549487b78e0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [79]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

/home/ranker/Desktop/Gen-AI/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [80]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [81]:
import bs4
loader = WebBaseLoader(
    web_path=("https://www.ypiyush.tech"),
    # bs_kwargs=dict(parse_only=bs4.SoupStrainer("p"))
)
docs= loader.load()
docs

[Document(metadata={'source': 'https://www.ypiyush.tech', 'title': 'Home | Piyush Yadav', 'description': 'Piyush Yadav is a full-stack developer in India. He is creative and passionate about design and technology so he always try to craft great-looking software products.', 'language': 'en-US'}, page_content='Home | Piyush YadavAboutGuestbookProjectsBlogContactSkip to main contentPiyush YadavStudent • Full-stack DeveloperI am a student and a Passionate and creative Full Stack developer based in India  🇮🇳SkillsNextJSReact NativeGolangJavascriptTypescriptHTML5CSS3TailwindCSSNodeJSMongoDBExpressJavaGraphQLGitPostgreSQLPrismaZustand/ RTKLinuxDockerTRPCFastAPIMySQLAzureJestRedisWork ExperienceCodemateSoftware Engineer - Full Stack Engineer - Oct 2023 - PresentLed the design and development of full-stack services for the Tech Innovation Group, enhancing system functionality and reliability using Next.js, Node.js, Langchain, Flask, Azure and Redis.Designed and implemented user-friendly dashboa

In [82]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vector_store=Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7549495bffa0>, search_kwargs={})

In [83]:
system_prompt= (
    "You are an assistant for question -answering tasks"
    "Use the following peices of retreived context to answer"
    "the question. if you don't know the answer, say that you "
    "don't know. Use three sentaces maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [84]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [87]:
response =rag_chain.invoke({"input": "Who is Piyush Yadav?"})

In [96]:
response["context"]

[Document(metadata={'description': 'Piyush Yadav is a full-stack developer in India. He is creative and passionate about design and technology so he always try to craft great-looking software products.', 'language': 'en-US', 'source': 'https://www.ypiyush.tech', 'title': 'Home | Piyush Yadav'}, page_content='Home | Piyush YadavAboutGuestbookProjectsBlogContactSkip to main contentPiyush YadavStudent • Full-stack DeveloperI am a student and a Passionate and creative Full Stack developer based in India  🇮🇳SkillsNextJSReact NativeGolangJavascriptTypescriptHTML5CSS3TailwindCSSNodeJSMongoDBExpressJavaGraphQLGitPostgreSQLPrismaZustand/ RTKLinuxDockerTRPCFastAPIMySQLAzureJestRedisWork ExperienceCodemateSoftware Engineer - Full Stack Engineer - Oct 2023 - PresentLed the design and development of full-stack services for the Tech Innovation Group, enhancing system functionality and reliability using Next.js, Node.js, Langchain, Flask, Azure and Redis.Designed and implemented user-friendly dashboa

# Addhing Chat History

In [92]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder


contextualize_system_prompt = (
    "given a chat history and the latest user question"
    "which might reference context in the chat history"
    "formulate a standalone question which can be understood"
    "without the chat history. Do NOT answer the question,"
    "just reformulate it if needed and otherwise return as as is. "
)
contextulaize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [93]:
history_aware_retreiver= create_history_aware_retriever(llm, retriever, contextulaize_q_prompt)
history_aware_retreiver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7549495bffa0>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChu

In [94]:
qa_prompt= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

In [95]:
question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retreiver, question_answer_chain)

In [98]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history = []
question="who is Piyush yadav?"
response1= rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Piyush is from which country?"
response2=rag_chain.invoke({"input": question2, "chat_history": chat_history})
print(response2["answer"])

Piyush Yadav is based in India.


In [99]:
chat_history

[HumanMessage(content='who is Piyush yadav?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Piyush Yadav is a student and a passionate full-stack developer based in India.', additional_kwargs={}, response_metadata={})]

In [100]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

conversation_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"   
)


In [103]:
conversation_rag_chain.invoke(
        {"input": "Who is Piyush Yadav and where he is from?"},
        config={"configurable": {"session_id": "piyush123"}},
 )["answer"]

conversation_rag_chain.invoke(
        {"input": "whose information i asked before?"},
        config={"configurable": {"session_id": "piyush123"}},
 )["answer"]

'You asked for information about a student and a Passionate and creative Full Stack developer based in India.'